## **3. Workflow Graph**

Defines how agents interact and specifies routing conditions.

In [ ]:
from langgraph.graph import StateGraph

# Create the workflow graph
graph = StateGraph(State)

# Add agents as nodes
graph.add_node("orchestrator_agent", orchestrator_agent)
graph.add_node("sql_agent", sql_agent)
graph.add_node("search_agent", search_agent)
graph.add_node("reporting_agent", reporting_agent)

# Define transitions between nodes
graph.add_edge("orchestrator_agent", "sql_agent", condition=lambda state: state.route == "sql")
graph.add_edge("orchestrator_agent", "search_agent", condition=lambda state: state.route == "search")
graph.add_edge("sql_agent", "orchestrator_agent")
graph.add_edge("search_agent", "orchestrator_agent")
graph.add_edge("orchestrator_agent", "reporting_agent", condition=lambda state: state.request_report)

# Define entry and finish points
graph.set_entry_point("orchestrator_agent")
graph.set_finish_point("orchestrator_agent")

# Compile the graph
app = graph.compile()

## **4. Run the Workflow**

### **Regular Query Without Report**

Handles a standard query where no report is requested.

In [ ]:
# Initial query without requesting a report
initial_state = State(user_query="Show me all customers in the database.")
final_state = app.invoke(initial_state)
print("\nFinal Response:", final_state.final_response)

### **Explicit Report Request**

Handles a follow-up query where the user requests a report.

In [ ]:
# User explicitly requests a report
report_request_state = State(
    user_query="Can you generate a report for this?",
    request_report=True,
    chat_history=final_state.chat_history
)
final_state = app.invoke(report_request_state)
print("\nFinal Response:", final_state.final_response)

## **5. Key Points**

- The **Orchestrator Agent** manages routing, aggregation, and finalization.  
- **SQL Agent** and **Search Agent** handle specialized tasks based on routing decisions.  
- The **Reporting Agent** builds a report from the final aggregated response.  
- **Chat history** tracks interactions across multiple user queries, providing shared context for all agents.

This setup balances clarity and flexibility, making it easy to maintain and extend.